- ### This .ipynb is a rule based approach for extracting ERM-Components from unstructured text

- ### Concept: 
    <img src="../pictures/workflow.png" width="500">
    
- ### Coding workflow:
    1. Preprocessing
        - Text cleaning
        - Fault correction (levenshtein distance) --> optional
    2. Structuring
        - Generate Subject-Predicate-Object tuples
        - Text Summarization (with term frequency-normalized) --> optional
        - Extract keywords (with tf-idf) --> optional
    3. Analysis
        - Extraction of primary keys
        - Extraction of attributes
        - Extraction of ISA-Relation --> optional (limitations)
        - Extraction of entities
        - Extraction of relations
        - Extraction of cardinalities
    4. Transformation
        - Generate .json output (to feed it into ER-Modeling Tool)


In [1]:
# necessary packages
import json
import re
import spacy
from spacy import displacy
from spellchecker import SpellChecker
from string import punctuation
import random
from spacy.lang.de.stop_words import STOP_WORDS
from heapq import nlargest
import os
from sklearn.feature_extraction.text import TfidfVectorizer


#### necessary pip installments ####
# pip install spacy
# pip install pyspellchecker
# python -m spacy download de_core_news_sm

### 1. Preprocessing

#### - Text cleaning (Step 1)

- main goal: transform text data from input.txt to preprocessedData.json
- the subfolder "examples" contains different ER-Diagram examples in unstructured text form

In [2]:
# get only text in "Text:"
# delete whitespaces and consider also text in next line etc.

input = "../examples/input.txt"
with open(input, 'r') as file:
    text = file.readlines()

language_line_index = None
for i, line in enumerate(text):
    if "Sprache:" in line:
        language_line_index = i
        break
    
cleaned_lines = [line.strip() for line in text[:language_line_index]]


single_line_text = ' '.join(cleaned_lines)

for line in text[language_line_index:]:
    single_line_text=single_line_text+"\n"+line
    
lines = single_line_text.split('\n')
cleaned_lines = [line.strip() for line in lines if line.strip()]

data = {}
for line in cleaned_lines:
    key, value = line.strip().split(': ',1)
    data[key] = value

text = data.get('Text', [])
data['Text'] = re.sub(r'\s+', ' ', text)

output = "../rule_based/preprocessedData.json"
with open(output, 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)


#### - Text cleaning (Step 2)

- find sentences in the text

In [3]:
# find sentences with spacy model (small)
# add new item to .json

with open(output, 'r', encoding='UTF-8') as f:
    data = json.load(f)

nlp = spacy.load('de_core_news_sm')

text = data.get('Text', [])
doc = nlp(text)
sentences = [sent.text.strip() for sent in doc.sents]
data["sentences"] = sentences

with open(output, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

#### - Text cleaning (Step 3)

- delete unneccessary sentences

In [4]:
# sentences with these words in it should not be considered
# these could be sentences for introduction of an exercise

with open(output, "r") as file:
    data = json.load(file)

filtered_text_list = []
for string in data["sentences"]:
    if not re.search(r"(Datenbank|ER-Modell|ER|Entität|Datensatz|SQL|relational|Modellierung)", string):
        filtered_text_list.append(string)
data["sentences_cleaned"] = filtered_text_list

with open(output, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

#### - Fault Correction

- pyspellchecker uses a Levenshtein Distance algorithm to find permutations within an edit distance of 1 from the original word.


In [5]:
# do not consider the spelling mistakes because also correct words are changed to incorrect words

with open(output, "r") as file:
    data = json.load(file)

spell = SpellChecker(language='de',distance=1) 

text = ' '.join(data.get('sentences_cleaned', []))
doc = nlp(text)
tokens = [token.text for token in doc]

misspelled = spell.unknown(tokens)

for word in misspelled:
    if spell.correction(word)!=None:
        tokens = [token.replace(word, spell.correction(word)) for token in tokens]
        ## UNCOMMENT FOR VISUALIZATION ###
        #print(word + " ---ersetzt durch---> " + spell.correction(word) + " ---Liste---> " + str(spell.candidates(word)))

punctuation = punctuation + '\n'
text = ''.join(" " + token if token not in punctuation and i > 0 else token for i, token in enumerate(tokens))
doc = nlp(text)
sentences = [sent.text.strip() for sent in doc.sents]  
data['sentences_corrected'] = sentences

with open(output, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

### 2. Structuring

#### - Term frequency - normalized (could be used for text-summerization)
Formula: 

<img src="../pictures/tf.png" width="500">

Concept:
1. Count occurrences per word in the text (stop words excluded)
2. Calculate weight per used word
3. Calculate sentence weight by summarizing weight per word
4. Find sentences with the highest weight

In [6]:
with open(output, "r") as file:
    data = json.load(file)

stopwords = list(STOP_WORDS)

# summerize only when more than tree sentences
if len(data["sentences_cleaned"]) > 3:
    # lower summerization factor when more than 15 sentences
    if len(data["sentences_cleaned"]) > 15:
        factor=0.3
    else:
        factor=0.5
    text = ' '.join(data.get('sentences_cleaned', []))
    doc = nlp(text)
    tokens = [token.text for token in doc]


    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    print(word_frequencies)

    max_frequency = max(word_frequencies.values())
    max_frequency

    # normalize frequency with maximal frequency
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency
    print(word_frequencies)

    sentence_tokens = [sent for sent in doc.sents]

    # count sentence frequency (sum of word frequency)
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]              
    print(sentence_scores)

    # maximal number of sentences for summerization
    select_length = int(len(sentence_tokens)*factor)
    print(str(select_length)+"="+str(len(sentence_tokens))+"*"+str(factor))
    
    # select most important sentences with highest sentence scores
    summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
    print(summary)

    final_summary = [word.text for word in summary]
    data['text_summerized'] = final_summary
    print(final_summary)


with open(output, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

{'Geschäft': 1, 'Filialen': 3, 'Filiale': 6, 'höchstens': 3, 'Filialleiter': 4, 'geführt': 1, 'leiten': 1, 'bestimmt': 1, 'bietet': 1, 'Produkte': 2, 'Produkt': 2, 'angeboten': 1, 'beschäftigt': 1, 'Mitarbeiter': 4, 'Verkauf': 3, 'bearbeiten': 1, 'involviert': 1, 'Jedes': 1, 'enthält': 2, 'Preis': 1, 'Produktname': 1, 'eindeutige': 1, 'Produkt-ID': 1, 'spezielle': 1, 'Produkttypen': 1, 'Neuwaren': 1, 'Verpackungsnummer': 1, 'registriert': 1, 'Gebrauchtwaren': 1, 'Anzahl': 1, 'Vorbesitzer': 1, 'mitgeführt': 1, 'Filial-ID': 1, 'eindeutig': 1, 'identifiziert': 2, 'Filialnamen': 1, 'Filialadresse': 1, 'Telefonnummer': 2, 'Datum': 1, 'Uhrzeit': 1, 'Betrag': 1, 'Mitarbeiternamen': 1, 'bestehend': 1, 'Vorname': 1, 'Nachname': 1, 'mehrere': 1, 'Mitarbeiteradressen': 1, 'ID': 1}
{'Geschäft': 0.16666666666666666, 'Filialen': 0.5, 'Filiale': 1.0, 'höchstens': 0.5, 'Filialleiter': 0.6666666666666666, 'geführt': 0.16666666666666666, 'leiten': 0.16666666666666666, 'bestimmt': 0.16666666666666666, 'b

#### - Term frequency - idf (could be used for keyword extraction)

- could be useful to identify entities later
- function TfidfVectorizer() is used for idf calculation

Formula: 

<img src="../pictures/idf2.png" width="500">
<img src="../pictures/idf1.png" width="500">

In [7]:
# generate collection.json 
# includes all input.txt
# used as document collection

def create_json_from_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.readlines()

    language_line_index = None
    for i, line in enumerate(text):
        if "Sprache:" in line:
            language_line_index = i
            break

    cleaned_lines = [line.strip() for line in text[:language_line_index]]
    single_line_text = ' '.join(cleaned_lines)

    for line in text[language_line_index:]:
        single_line_text = single_line_text + "\n" + line

    lines = single_line_text.split('\n')
    cleaned_lines = [line.strip() for line in lines if line.strip()]

    data = {}
    for line in cleaned_lines:
        key, value = line.strip().split(': ', 1)
        data[key] = value

    text = data.get('Text', [])
    data['Text'] = re.sub(r'\s+', ' ', text)

    return data


folder_path = "../examples"
file_list = os.listdir(folder_path)

all_data = []
for file_name in file_list:
    if file_name.endswith(".txt"): 
        file_path = os.path.join(folder_path, file_name)
        json_data = create_json_from_text_file(file_path)
        all_data.append(json_data)

output_collection = "../rule_based/collection.json"
with open(output_collection, 'w') as file:
    json.dump(all_data, file, indent=4, ensure_ascii=False)

In [8]:
# get keywords with "inverted document frequency" for each input text
# saved in collection.json

with open(output_collection, "r") as file:
    data = json.load(file)

docs=[]
for item in data:
    docs.append(item.get('Text', []))
# german vowels --> limitation (siehe collection.json)
tv = TfidfVectorizer()
t_vec = tv.fit_transform(docs)

for doc_id in range(0,len(docs)):
    my_list = []
    for i in range(0,t_vec.shape[1]):
        # dont consider numbers
        if t_vec[doc_id,i] and not tv.get_feature_names_out()[i].isnumeric():
            my_list.append({"Wort": tv.get_feature_names_out()[i], "Gewicht" : t_vec[doc_id,i]})
    my_list = sorted(my_list, key=lambda x: -x["Gewicht"])
    # only the three most important keywords
    data[doc_id]['keywords']=my_list[:3]

with open(output_collection, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

#### - Generate Subject-Verb-Object tuples

In [9]:
doc = nlp("Ein Geschäft hat viele Filialen.")
displacy.render(doc, style="dep", jupyter=True)

In [10]:
# spo sentences
# saved in preprocessedData.json

def s_p_o(doc):
    res = []
    for verb in [d for d in doc if d.pos_ == "VERB"]:
        sb = [c.lemma_ for c in verb.children if c.dep_ == "sb"]
        ob = [c.lemma_ for c in verb.children if c.dep_ == "oa"]
        if len(sb) == 1 and len(ob) == 1:
            res.append([sb[0], verb, ob[0]])
    return res

output = "../rule_based/preprocessedData.json"
with open(output, "r") as file:
    data = json.load(file)

svo_list = []
text = data.get('sentences_cleaned', [])
for sent in text:
    doc = nlp(sent)
    svo = s_p_o(doc)
    if len(svo)==1:
        text = svo[0][0] + ''.join(" " + str(token) for token in svo[0][1:])  
        svo_list.append(text)


data['text_spo'] = svo_list

with open(output, "w") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

### 3. Analysis

#### - Extraction of primary keys

In [11]:
doc = nlp("Jedes Produkt enthält Preis, Produktname und eindeutige Produkt-ID.")
displacy.render(doc, style="dep", jupyter=True)

In [12]:
# get all nouns and than filter prim keys
# prim keys: ...-ID or ...-Nummer --> limitation

with open(output, "r") as file:
    data = json.load(file)
    
sentences = data["sentences_cleaned"]
nouns_list = []

for sentence in sentences:
    doc = nlp(sentence)
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    nouns_list.extend(nouns)

nouns_list_lower = [word.lower() for word in nouns_list]
filtered_nouns_list = [word for word in nouns_list_lower if re.search(r'\b-id\b', word) or re.search(r'\b-nr\b', word)]
word_to_remove = "id"
filtered_pk_list = [word for word in filtered_nouns_list if word != word_to_remove]
print(filtered_pk_list)

['produkt-id', 'filial-id']


In [13]:
# if sentence contains only the word "eindeutig" than the last noun should be prim key with "noun-ID"

result_list = []
for sentence in sentences:
    doc = nlp(sentence)
    found_noun = None
    for token in doc:
        if "eindeutig" in token.text.lower():
            for token in reversed(doc):
                    if token.pos_ == "NOUN":
                        found_noun = token.text
                        break
    
    if found_noun:
        if "-ID" not in found_noun:
            result_list.append(found_noun + "-ID")
        else:
             result_list.append(found_noun)
             
print(result_list)

['Produkt-ID', 'Filial-ID']


In [14]:
# when only the words "ID" or "id" occur --> "noun-ID"

result_list1 = []
for sentence in sentences:
    doc = nlp(sentence)
    found_nouns = []
    for token in doc:
        if token.text.lower() == "id":
            for token in reversed(doc):
                if token.pos_ == "NOUN" and token.text.lower() != "id" and token.text not in found_nouns:
                    found_nouns.append(token.text)

result_list1.extend([noun + "-ID" for noun in found_nouns])      
print(result_list1)

['Filialleiter-ID', 'Mitarbeiter-ID']


In [15]:
# final pk list
final_pk = filtered_pk_list + result_list + result_list1
lowercase_word_list = [word.lower() for word in final_pk]
# delete duplicate with set()
final_pk = list(set(lowercase_word_list))
print(final_pk)

['filialleiter-id', 'filial-id', 'mitarbeiter-id', 'produkt-id']


#### - Extraction of attributes

In [16]:
# if more then two nouns in sentence: first=entity, other=attributes

prefinal_attr_list = []
attr_list = []
for sentence in sentences:
    doc = nlp(sentence)
    found_nouns = []
    for token in doc:
        if token.pos_ == "NOUN":
            found_nouns.append(token.text.lower())

    if len(found_nouns)>2:
        prefinal_attr_list.append([found_nouns[0],found_nouns[1:]])
        attr_list.extend(found_nouns[1:])

#print(prefinal_attr_list)

final_attr_list = [word for word in attr_list if '-id' not in word and 'id' != word]
print(final_attr_list)

def remove_id_words(lst):
    new_list = []
    for item in lst:
        if isinstance(item, list):
            new_sublist = [subitem for subitem in item[1] if not ('id' in subitem or '-id' in subitem)]
            new_list.append([item[0], new_sublist])
    return new_list

final_attr_list_ents = remove_id_words(prefinal_attr_list)
print(final_attr_list_ents)

['preis', 'produktname', 'anzahl', 'vorbesitzer', 'filialnamen', 'filialadresse', 'telefonnummer', 'datum', 'uhrzeit', 'betrag', 'mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer', 'filialleiter']
[['produkt', ['preis', 'produktname']], ['gebrauchtwaren', ['anzahl', 'vorbesitzer']], ['filiale', ['filialnamen', 'filialadresse', 'telefonnummer']], ['verkauf', ['datum', 'uhrzeit', 'betrag']], ['mitarbeiter', ['mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer']], ['mitarbeiter', ['filialleiter']]]


#### - Extraction of ISA-Relation

In [17]:
# if sentence contains following verbs than it is a generalization

isa_list1 = []
target_verbs = ["einbeziehen", "bestehen", "umfassen", "teilen", "beinhalten"]
for sentence in sentences:
    doc = nlp(sentence)
    found_verbs = set()
    for token in doc:
        if token.lemma_ in target_verbs:
            found_verbs.add(token.lemma_)
    if found_verbs:
        found_nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        isa_list1.extend(found_nouns)

# make distinct
isa_list1 = list(set(isa_list1))

print(isa_list1)

[]


In [18]:
# sentence contain "typ" as word (noun) --> limitation: order can be different, maybe two splitted sentences etc.

for sentence in sentences:
    doc = nlp(sentence)
    found_nouns = []
    isa_list2 = []
    for token in doc:
        if token.text.lower() == "typ":
            for token in doc:
                if token.pos_ == "NOUN":
                    found_nouns.append(token.text)
            for token in doc:
                if token.pos_ == "NOUN" and token.text != found_nouns[0]:
                    isa_list2.append(token.text)

print(isa_list2)

[]


In [19]:
final_isa_list = []
final_isa_list.extend(isa_list1)
final_isa_list.extend(isa_list2)
print(final_isa_list)

[]


#### - Extraction of entities

In [20]:
# filtern aus svo list wenn in den vorherigen beiden lsiten das wort drin ist
with open(output, "r") as file:
    data = json.load(file)

spo = data["text_spo"]
first_last_words = []
for sentence in spo:
    doc = nlp(sentence)
    first_word = doc[0].text.lower()
    last_word = doc[-1].text.lower()
    first_last_words.append(first_word)
    first_last_words.append(last_word)

unique_first_last_words = list(set(first_last_words))

print(unique_first_last_words)

# limitation: alle attribute müssen verschieden heißen: zwei mal name geht nicht 
# diese ents - att&pk_list = nur_ents_diese_list
att_pk_list = []
att_pk_list.extend(final_attr_list)
att_pk_list.extend(final_pk)
print(att_pk_list)

final_ents =[]
final_ents = [word for word in unique_first_last_words if word not in att_pk_list]
print(final_ents)

['verkauf', 'preis', 'mitarbeiter', 'datum', 'filiale', 'geschäft', 'filialnamen', 'produkt']
['preis', 'produktname', 'anzahl', 'vorbesitzer', 'filialnamen', 'filialadresse', 'telefonnummer', 'datum', 'uhrzeit', 'betrag', 'mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer', 'filialleiter', 'filialleiter-id', 'filial-id', 'mitarbeiter-id', 'produkt-id']
['verkauf', 'mitarbeiter', 'filiale', 'geschäft', 'produkt']


#### - Extraction of relations

In [21]:
# relations from spo sentences
relations = []
for sentence in spo:
    doc = nlp(sentence)
    if doc[0].text.lower() in final_ents and doc[-1].text.lower() in final_ents:
        relations.append([doc[0].text.lower(),doc[1].text, doc[-1].text.lower()])

print(relations)

[['geschäft', 'hat', 'filiale'], ['filiale', 'bietet', 'produkt'], ['filiale', 'beschäftigt', 'mitarbeiter']]


In [22]:
doc = nlp("Im Verkauf können viele Produkte involviert sein.")
displacy.render(doc, style="dep", jupyter=True)

In [23]:
# if one sentence has two nouns (no attribute or ISA-beziehung or primary keys) and one verb than the verb can be a relation

relations2 = []
save_found_noun_pairs = []
for sentence in sentences:
    doc = nlp(sentence)
    found_nouns = []
    for token in doc:
        if token.pos_ == "NOUN":
            found_nouns.append(token.lemma_.lower())
        if token.pos_ == "VERB":
            verb = token.text
    if len(found_nouns)==2:
        if all(word not in final_attr_list and word not in final_isa_list and word not in final_pk for word in found_nouns):
            if found_nouns not in save_found_noun_pairs:
                save_found_noun_pairs.append(found_nouns)
                save_found_noun_pairs.append([found_nouns[1],found_nouns[0]])
                relations2.append([found_nouns[0],verb,found_nouns[1]])

print(relations2)

[['geschäft', 'hat', 'filiale'], ['filiale', 'bietet', 'produkt'], ['filiale', 'beschäftigt', 'mitarbeiter'], ['mitarbeiter', 'bearbeiten', 'verkauf'], ['verkauf', 'involviert', 'produkt'], ['produkttype', 'registriert', 'verpackungsnummer']]


In [24]:
relations2.extend(relations)

final_beziehungen = []
for sublist in relations2:
    if sublist not in final_beziehungen:
        final_beziehungen.append(sublist)

print(final_beziehungen)

[['geschäft', 'hat', 'filiale'], ['filiale', 'bietet', 'produkt'], ['filiale', 'beschäftigt', 'mitarbeiter'], ['mitarbeiter', 'bearbeiten', 'verkauf'], ['verkauf', 'involviert', 'produkt'], ['produkttype', 'registriert', 'verpackungsnummer']]


#### - Extraction of cardinalities

In [25]:
doc = nlp("Ein Geschäft hat viele Filialen.")
displacy.render(doc, style="dep", jupyter=True)

In [26]:
doc = nlp("Jede Filiale darf von höchstens einem Filialleiter geführt werden.")
displacy.render(doc, style="dep", jupyter=True)

In [27]:
doc = nlp("Im Verkauf können viele Produkte involviert sein.")
displacy.render(doc, style="dep", jupyter=True)

In [28]:
# get indices for each spo
def find_matching_sentence(teilliste, saetze):
    for i, satz in enumerate(saetze):
        if all(wort in satz.lower() for wort in teilliste):
            return i
    return None

index_list=[]
for teilliste in final_beziehungen:
    index = find_matching_sentence(teilliste, sentences)
    if index is not None:
        index_list.append([teilliste,index])

print(index_list)

[[['geschäft', 'hat', 'filiale'], 0], [['filiale', 'bietet', 'produkt'], 4], [['filiale', 'beschäftigt', 'mitarbeiter'], 6], [['mitarbeiter', 'bearbeiten', 'verkauf'], 7], [['verkauf', 'involviert', 'produkt'], 8], [['produkttype', 'registriert', 'verpackungsnummer'], 10]]


In [29]:
# convert det to min max cardinality

def get_determiners_for_noun(index):
    doc = nlp(sentences[index])
    noun_det_pairs = []
    for i, token in enumerate(doc):
        if token.dep_ == "nk": 
            noun = token.head.text 
            det = token.text
            prev_word = doc[i-1].text if i > 0 else None
            min_val, max_val = convert_det_to_min_max(det, prev_word)
            noun_det_pairs.append((index, noun, prev_word, det, min_val, max_val))
    return noun_det_pairs

def convert_det_to_min_max(det, prev_word):
    if "mindestens" == prev_word:
        return 1, "n"
    elif "höchstens" == prev_word:
        return 0, 1
    elif "ein" in det.lower():
        return 1, 1
    elif "viele" in det.lower():
        return 1, "n"
    elif "jede" in det.lower():
        return 1, 1
    elif "genau" in det.lower():
        return 1, 1
    else:
        return "n", "m"
    
result_cardinality = []
for item in index_list:
    index = item[1]
    result = get_determiners_for_noun(index)
    result_cardinality.append(result)

print(result_cardinality)

[[(0, 'Geschäft', None, 'Ein', 1, 1), (0, 'Filialen', 'hat', 'viele', 1, 'n')], [(4, 'Filiale', None, 'Die', 'n', 'm'), (4, 'Produkte', 'bietet', 'viele', 1, 'n')], [(6, 'Filiale', None, 'Die', 'n', 'm'), (6, 'Mitarbeiter', 'beschäftigt', 'viele', 1, 'n')], [(7, 'Mitarbeiter', None, 'Ein', 1, 1), (7, 'Verkauf', 'höchstens', 'einen', 0, 1)], [(8, 'Im', 'Im', 'Verkauf', 'n', 'm'), (8, 'Produkte', 'können', 'viele', 1, 'n')], [(10, 'Produkttypen', 'gibt', 'zwei', 'n', 'm'), (10, 'Produkttypen', 'zwei', 'spezielle', 'n', 'm'), (10, 'bei', 'bei', 'denen', 'n', 'm'), (10, 'Verpackungsnummer', 'denen', 'die', 'n', 'm')]]


In [30]:
# change the cardinality for each entity with the corresponding entity for correct min max notation

def tausche_elemente(liste):
    newlist=[]
    index_count = {}
    for tupel in liste:
        index = tupel[0][0]
        if index in index_count:
            index_count[index].append(tupel)
        else:
            index_count[index] = [tupel]
    for index, tupel_list in index_count.items():
        if len(tupel_list[0]) == 2:
            tupel1, tupel2 = tupel_list[0]
            neue_tupel1 = tupel1[:-2] + (tupel2[-2], tupel2[-1])
            neue_tupel2 = tupel2[:-2] + (tupel1[-2], tupel1[-1])
            newlist.append(neue_tupel1)
            newlist.append(neue_tupel2)

    return newlist


final_card = tausche_elemente(result_cardinality)
print(final_card)

[(0, 'Geschäft', None, 'Ein', 1, 'n'), (0, 'Filialen', 'hat', 'viele', 1, 1), (4, 'Filiale', None, 'Die', 1, 'n'), (4, 'Produkte', 'bietet', 'viele', 'n', 'm'), (6, 'Filiale', None, 'Die', 1, 'n'), (6, 'Mitarbeiter', 'beschäftigt', 'viele', 'n', 'm'), (7, 'Mitarbeiter', None, 'Ein', 0, 1), (7, 'Verkauf', 'höchstens', 'einen', 1, 1), (8, 'Im', 'Im', 'Verkauf', 1, 'n'), (8, 'Produkte', 'können', 'viele', 'n', 'm')]


In [ ]:
# in one sentence maximal one relation --> limitation
# cardinality with 2 or 3 not in available --> limitation

In [31]:
# bring id into final_attr_list_ents (reason: assignement info for id to a specific entity)

def extract_and_insert_id_words(list1, list2):
    for word1 in list1:
        for idx, sublist2 in enumerate(list2):
            outer_word = sublist2[0]
            inner_words = sublist2[1]
            for word2 in inner_words:
                if (word1.split('-')[0]) in outer_word:
                    sublist2[1].append(word1)
                    break
    return list2

result = extract_and_insert_id_words(final_pk, final_attr_list_ents)
print(result)

[['produkt', ['preis', 'produktname', 'produkt-id']], ['gebrauchtwaren', ['anzahl', 'vorbesitzer']], ['filiale', ['filialnamen', 'filialadresse', 'telefonnummer', 'filial-id']], ['verkauf', ['datum', 'uhrzeit', 'betrag']], ['mitarbeiter', ['mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer', 'mitarbeiter-id']], ['mitarbeiter', ['filialleiter', 'mitarbeiter-id']]]


In [32]:
# Remove duplicates from inner_words

def merge_inner_words(list_data):
    merged_data = {}
    for outer_word, inner_words in list_data:
        if outer_word in merged_data:
            merged_data[outer_word].extend(inner_words)
        else:
            merged_data[outer_word] = inner_words

    for outer_word, inner_words in merged_data.items():
        merged_data[outer_word] = list(set(inner_words))

    merged_list = [[outer_word, inner_words] for outer_word, inner_words in merged_data.items()]
    return merged_list

final_attr_and_id_list_ents = merge_inner_words(result)
print(final_attr_and_id_list_ents)

[['produkt', ['produktname', 'preis', 'produkt-id']], ['gebrauchtwaren', ['vorbesitzer', 'anzahl']], ['filiale', ['filialnamen', 'filial-id', 'telefonnummer', 'filialadresse']], ['verkauf', ['betrag', 'datum', 'uhrzeit']], ['mitarbeiter', ['nachname', 'mitarbeiteradressen', 'mitarbeiter-id', 'mitarbeiternamen', 'filialleiter', 'vorname', 'telefonnummer']]]


In [33]:
print(final_ents)
print("--------------")
print(final_pk)
print("--------------")
print(final_attr_list)
print("--------------")
print(final_attr_list_ents)
print("--------------")
print(final_attr_and_id_list_ents)
print("--------------")
print(final_isa_list)
print("--------------")
print(index_list)
print("--------------")
print(final_card)

['verkauf', 'mitarbeiter', 'filiale', 'geschäft', 'produkt']
--------------
['filialleiter-id', 'filial-id', 'mitarbeiter-id', 'produkt-id']
--------------
['preis', 'produktname', 'anzahl', 'vorbesitzer', 'filialnamen', 'filialadresse', 'telefonnummer', 'datum', 'uhrzeit', 'betrag', 'mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer', 'filialleiter']
--------------
[['produkt', ['preis', 'produktname', 'produkt-id']], ['gebrauchtwaren', ['anzahl', 'vorbesitzer']], ['filiale', ['filialnamen', 'filialadresse', 'telefonnummer', 'filial-id']], ['verkauf', ['datum', 'uhrzeit', 'betrag']], ['mitarbeiter', ['mitarbeiternamen', 'vorname', 'nachname', 'mitarbeiteradressen', 'telefonnummer', 'mitarbeiter-id', 'filialleiter', 'mitarbeiter-id']], ['mitarbeiter', ['filialleiter', 'mitarbeiter-id']]]
--------------
[['produkt', ['produktname', 'preis', 'produkt-id']], ['gebrauchtwaren', ['vorbesitzer', 'anzahl']], ['filiale', ['filialnamen', 'filial-id', 'telefonnummer

### 4. Transformation

In [34]:
# insert entities into template

template_json = '''
{
  "projectVersion": 1,
  "projectName": "notNamed",
  "erContent": {
    "drawBoardContent": {
      "drawBoardElements": [],
      "connections": []
    }
  },
  "relContent": {
    "drawBoardContent": {
      "tables": [],
      "connections": []
    }
  }
}
'''

string_list = final_ents

def generate_unique_id(prefix):
    return f"{prefix}--{random.randint(1000000000000, 9999999999999)}"

data = json.loads(template_json)

for name in string_list:
    unique_id = generate_unique_id("StrongEntity")
    draw_board_element = {
        "id": unique_id,
        "displayName": name,
        "isHighlighted": False,
        "isSelected": False,
        "x": 0,
        "y": 0,
        "width": 151.015625,
        "height": 67,
        "objectType": "DrawBoardElement",
        "erType": "StrongEntity",
        "owningSide": None
    }
    data["erContent"]["drawBoardContent"]["drawBoardElements"].append(draw_board_element)

filled_json = json.dumps(data, indent=2)

final_output = "../rule_based/output.json"
with open(final_output, "w") as file:
    file.write(filled_json)


In [35]:
# insert attributes/id and attribute/id-connection to entity into template json file

with open(final_output, 'r', encoding='UTF-8') as f:
    data = json.load(f)

for inner_list in final_attr_and_id_list_ents:
    entity_name, attributes = inner_list[0], inner_list[1]
    if entity_name in final_ents:
        for attribute_name in attributes:
            if "-id" in attribute_name:
                unique_id = generate_unique_id("IdentifyingAttribute")
                draw_board_element = {
                "id": unique_id,
                "displayName": attribute_name,
                "isHighlighted": False,
                "isSelected": False,
                "x": 0,
                "y": 0,
                "width": 222,
                "height": 70,
                "objectType": "DrawBoardElement",
                "erType": "IdentifyingAttribute",
                "owningSide": None
                }
                data["erContent"]["drawBoardContent"]["drawBoardElements"].append(draw_board_element)
            else:
                unique_id = generate_unique_id("NormalAttribute")
                draw_board_element = {
                    "id": unique_id,
                    "displayName": attribute_name,
                    "isHighlighted": False,
                    "isSelected": False,
                    "x": 0,
                    "y": 0,
                    "width": 222,
                    "height": 70,
                    "objectType": "DrawBoardElement",
                    "erType": "NormalAttribute",
                    "owningSide": None
                }
                data["erContent"]["drawBoardContent"]["drawBoardElements"].append(draw_board_element)

            for existing_element in data["erContent"]["drawBoardContent"]["drawBoardElements"]:
                if existing_element["displayName"] == entity_name:
                    connection_id = generate_unique_id("Connection")
                    connection_block = {
                        "id": f"{unique_id} --> {existing_element['id']} - {connection_id}",
                        "start": unique_id,
                        "end": existing_element['id'],
                        "min": "1",
                        "max": "1",
                        "objectType": "Connection",
                        "isSelected": False,
                        "withArrow": False,
                        "withLabel": False,
                        "connectionType": "AttributeConnector",
                        "isHighlighted": False
                    }
                    data["erContent"]["drawBoardContent"]["connections"].append(connection_block)

filled_json = json.dumps(data, indent=2)

with open(final_output, "w") as file:
    file.write(filled_json)

In [69]:
# limitation: if entity has no id --> must correct it in editor tool

In [36]:
# prepare list for relations between entities (map cardinality to entities with relation)

mapping_dict = {}

for item in index_list:
    key = item[1] 
    value = item[0]
    mapping_dict[key] = value

result_list = []

for tup in final_card:
    key = tup[0] 
    value_from_dict = mapping_dict.get(key, None)
    if value_from_dict is not None:
        result_list.append((tup, value_from_dict))

print(result_list)

[((0, 'Geschäft', None, 'Ein', 1, 'n'), ['geschäft', 'hat', 'filiale']), ((0, 'Filialen', 'hat', 'viele', 1, 1), ['geschäft', 'hat', 'filiale']), ((4, 'Filiale', None, 'Die', 1, 'n'), ['filiale', 'bietet', 'produkt']), ((4, 'Produkte', 'bietet', 'viele', 'n', 'm'), ['filiale', 'bietet', 'produkt']), ((6, 'Filiale', None, 'Die', 1, 'n'), ['filiale', 'beschäftigt', 'mitarbeiter']), ((6, 'Mitarbeiter', 'beschäftigt', 'viele', 'n', 'm'), ['filiale', 'beschäftigt', 'mitarbeiter']), ((7, 'Mitarbeiter', None, 'Ein', 0, 1), ['mitarbeiter', 'bearbeiten', 'verkauf']), ((7, 'Verkauf', 'höchstens', 'einen', 1, 1), ['mitarbeiter', 'bearbeiten', 'verkauf']), ((8, 'Im', 'Im', 'Verkauf', 1, 'n'), ['verkauf', 'involviert', 'produkt']), ((8, 'Produkte', 'können', 'viele', 'n', 'm'), ['verkauf', 'involviert', 'produkt'])]


In [37]:
# created new list for adding relation to template

extracted_info = []

for index, tupel in enumerate(result_list):
    inner_tupel, word_list = tupel
    beziehung = word_list[1]
    if index % 2 == 0:
        wort = word_list[0]
        info = (wort, beziehung, str(inner_tupel[-2]), str(inner_tupel[-1]))
    else:
        wort = word_list[2]
        info = (wort, beziehung, str(inner_tupel[-2]), str(inner_tupel[-1]))
        
    extracted_info.append(info)

print(extracted_info)
print("---------------------")
for info in extracted_info:
    print(f"Entität '{info[0]}': Beziehung = '{info[1]}',  min = '{info[2]}', max = '{info[3]}'")

[('geschäft', 'hat', '1', 'n'), ('filiale', 'hat', '1', '1'), ('filiale', 'bietet', '1', 'n'), ('produkt', 'bietet', 'n', 'm'), ('filiale', 'beschäftigt', '1', 'n'), ('mitarbeiter', 'beschäftigt', 'n', 'm'), ('mitarbeiter', 'bearbeiten', '0', '1'), ('verkauf', 'bearbeiten', '1', '1'), ('verkauf', 'involviert', '1', 'n'), ('produkt', 'involviert', 'n', 'm')]
---------------------
Entität 'geschäft': Beziehung = 'hat',  min = '1', max = 'n'
Entität 'filiale': Beziehung = 'hat',  min = '1', max = '1'
Entität 'filiale': Beziehung = 'bietet',  min = '1', max = 'n'
Entität 'produkt': Beziehung = 'bietet',  min = 'n', max = 'm'
Entität 'filiale': Beziehung = 'beschäftigt',  min = '1', max = 'n'
Entität 'mitarbeiter': Beziehung = 'beschäftigt',  min = 'n', max = 'm'
Entität 'mitarbeiter': Beziehung = 'bearbeiten',  min = '0', max = '1'
Entität 'verkauf': Beziehung = 'bearbeiten',  min = '1', max = '1'
Entität 'verkauf': Beziehung = 'involviert',  min = '1', max = 'n'
Entität 'produkt': Beziehu

In [72]:
# write relations into template

with open(final_output, 'r', encoding='UTF-8') as f:
    data = json.load(f)

def generate_unique_id2():
    return f"{random.randint(1000000000000, 9999999999999)}"

def create_strong_relations(json_data, relations_list):
    for i, relation in enumerate(relations_list):
        if i % 2 == 1: 
            display_name = relation[1]
            unique_id = generate_unique_id("StrongRelation")
            strong_relation_element = {
                "id": unique_id,
                "displayName": display_name,
                "isHighlighted": False,
                "isSelected": False,
                "x": 0,
                "y": 0,
                "width": 151.015625,
                "height": 100.67708333333333,
                "objectType": "DrawBoardElement",
                "erType": "StrongRelation",
                "owningSide": None
            }
            json_data["erContent"]["drawBoardContent"]["drawBoardElements"].append(strong_relation_element)

            # Search StrongEntity-Elements with the "displayName" to get their id in the template
            for existing_element in data["erContent"]["drawBoardContent"]["drawBoardElements"]:

                if existing_element["displayName"] == relation[0]:
                    entity_id_1 = existing_element["id"]
                    connection_id_1 = generate_unique_id2()
                    connection_block_1 = {
                        "id": f"{unique_id} --> {entity_id_1} - {connection_id_1}",
                        "start": unique_id,
                        "end": entity_id_1,
                        "min": relation[2],
                        "max": relation[3],
                        "objectType": "Connection",
                        "isSelected": False,
                        "withArrow": False,
                        "withLabel": True,
                        "connectionType": "Association",
                        "isHighlighted": False
                    }
                    json_data["erContent"]["drawBoardContent"]["connections"].append(connection_block_1)
                    
            for existing_element in data["erContent"]["drawBoardContent"]["drawBoardElements"]:
                if existing_element["displayName"] == relations_list[i-1][0]:
                    entity_id_2 = existing_element["id"]
                    connection_id_2 = generate_unique_id2()
                    connection_block_2 = {
                        "id": f"{entity_id_2} --> {unique_id} - {connection_id_2}",
                        "start": entity_id_2,
                        "end": unique_id,
                        "min": relations_list[i-1][2],
                        "max": relations_list[i-1][3],
                        "objectType": "Connection",
                        "isSelected": False,
                        "withArrow": False,
                        "withLabel": True,
                        "connectionType": "Association",
                        "isHighlighted": False
                    }
                    json_data["erContent"]["drawBoardContent"]["connections"].append(connection_block_2)

    return json_data

data = create_strong_relations(data, extracted_info)
filled_json = json.dumps(data, indent=2)

with open(final_output, "w") as file:
    file.write(filled_json)